# 0.0 Imports

## 0.1 Libraries

In [2]:
import pandas as pd
import numpy as np
import inflection
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## 0.2 Data

In [44]:
dados_sales = pd.read_csv('/Users/nando/Comunidade DS/ds_em_producao/data/train.csv',low_memory=False)
dados_stores = pd.read_csv('/Users/nando/Comunidade DS/ds_em_producao/data/store.csv')
dados = pd.merge(dados_sales,dados_stores,how='left',on='Store')
dados = dados.sample(int(round(dados.shape[0]/10,0)),random_state=101) # Reduce the size of the dataset in 10 (Too big)
dados.reset_index(drop=True,inplace=True)
dados.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,965,7,2014-11-02,0,0,0,0,0,0,a,c,110.0,NaN,NaN,0,NaN,NaN,NaN
1,268,1,2013-12-30,6995,814,1,0,0,1,a,a,4520.0,2.0,2014.0,0,NaN,NaN,NaN
2,1112,6,2015-03-28,8671,774,1,0,0,0,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
3,128,1,2014-07-21,6675,534,1,0,0,0,d,c,2000.0,NaN,NaN,1,1.0,2013.0,"Jan,Apr,Jul,Oct"
4,687,4,2015-06-11,6961,675,1,0,0,0,d,c,2770.0,NaN,NaN,0,NaN,NaN,NaN


## 0.3 Helper Functions

# 1.0 Data Description and Cleaning

## 1.1 Rename Columns and Values

In [45]:
# CamelCase to snake_case
for c in range(len(dados.columns)):
    dados.rename(columns={dados.columns.values[c]:inflection.underscore(dados.columns.values[c])},inplace=True)

In [46]:
dados['state_holiday'] = dados['state_holiday'].map({'a':'public holiday', 'b':'Easter holiday', 'c':'Christmas', '0':'None'})
dados['assortment'] = dados['assortment'].map({'a':'basic','b':'extra','c':'extended'})

## 1.2 Data Dimension

In [47]:
print(f'Rows: {dados.shape[0]}\nColumns: {dados.shape[1]}')

Rows: 101721
Columns: 18


## 1.3 Data Types

### 1.3.1 Check

In [48]:
dados.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

### 1.3.2 Change

In [49]:
# date to datetime64
dados['date'] = pd.to_datetime(dados['date'])

## 1.4 NaN

### 1.4.1 Check

In [50]:
dados.isna().sum()

store                               0
day_of_week                         0
date                                0
sales                               0
customers                           0
open                                0
promo                               0
state_holiday                       0
school_holiday                      0
store_type                          0
assortment                          0
competition_distance              257
competition_open_since_month    32411
competition_open_since_year     32411
promo2                              0
promo2_since_week               50900
promo2_since_year               50900
promo_interval                  50900
dtype: int64

### 1.4.2 Dealing With

In [51]:
## competition_distance, competition_open_since_month, competition_open_since_year

# I suppose that the competition_distance == nan refers that there's no competition!
# Lets'se if it's true

dados[dados['competition_distance'].isna()].isna().sum()

store                             0
day_of_week                       0
date                              0
sales                             0
customers                         0
open                              0
promo                             0
state_holiday                     0
school_holiday                    0
store_type                        0
assortment                        0
competition_distance            257
competition_open_since_month    257
competition_open_since_year     257
promo2                            0
promo2_since_week               179
promo2_since_year               179
promo_interval                  179
dtype: int64

Yeah, thats true. 

All of the empty distances have empty competition_open_since.. what confirm that the competition has never been started! 

In [52]:
for index in dados[dados['competition_distance'].isna()].index:
    dados.loc[index,'competition_distance'] = 999999 # High values suggests that the competitors are so far that there's no competition
    dados.loc[index,'competition_open_since_month'] = 0 # It suggets that it has never been opened
    dados.loc[index,'competition_open_since_year'] = 0 # It suggets that it has never been opened

In [53]:
## competition_open_since_month, competition_open_since_year

# I suppose that if there's no year, there's no month and vice-versa
# Let's se if it's true

dados[dados['competition_open_since_month'].isna()].isna().sum()

store                               0
day_of_week                         0
date                                0
sales                               0
customers                           0
open                                0
promo                               0
state_holiday                       0
school_holiday                      0
store_type                          0
assortment                          0
competition_distance                0
competition_open_since_month    32154
competition_open_since_year     32154
promo2                              0
promo2_since_week               13746
promo2_since_year               13746
promo_interval                  13746
dtype: int64

Yeah, 100% true!

Then, i suppose that we do not have these dates by some reason

In [54]:
month_median = round(dados['competition_open_since_month'].median(),0)
year_median = round(dados['competition_open_since_year'].median(),0)
for index in dados[dados['competition_open_since_month'].isna()].index:
    dados.loc[index,'competition_open_since_month'] = month_median # Im gonna use median and round to have discret values
    dados.loc[index,'competition_open_since_year'] = year_median # Im gonna use median and round to have discret values

In [55]:
## promo_interval, promo2_since_week, promo2_since_year

# I wanna check if the NaN in these columns are just when promo2 is 0 (no promo2) what would be very logic.
# Because if there's no promo2, theres no since date or interval

(dados[dados['promo2']==0].isna()).sum()

store                               0
day_of_week                         0
date                                0
sales                               0
customers                           0
open                                0
promo                               0
state_holiday                       0
school_holiday                      0
store_type                          0
assortment                          0
competition_distance                0
competition_open_since_month        0
competition_open_since_year         0
promo2                              0
promo2_since_week               50900
promo2_since_year               50900
promo_interval                  50900
dtype: int64

So true! 

Every NaN in the last 3 columns is because promo2 has never been started!

In [56]:
for index in dados[dados['promo2_since_week'].isna()].index:
    dados.loc[index, 'promo2_since_week'] = 0 # It suggets that there's no promo2 week start
    dados.loc[index, 'promo2_since_year'] = 0 # It suggets that there's no promo2 year start
    dados.loc[index, 'promo_interval'] = 'No promo2' # For now, im going to substitute with this

### 1.4.3 Final Check

In [57]:
dados.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
dtype: int64

## 1.5 Data Types (after NaN)

### 1.5.1 Check

In [58]:
dados.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

### 1.5.2 Change

In [59]:
# float to int in columns related to dates
dados['competition_open_since_month'] = dados['competition_open_since_month'].astype(int)
dados['competition_open_since_year'] = dados['competition_open_since_year'].astype(int)
dados['promo2_since_week'] = dados['promo2_since_week'].astype(int)
dados['promo2_since_year'] = dados['promo2_since_year'].astype(int)

## 1.6 Descriptive Statistics

### 1.6.1 Separate Categorical and Numeric Variables

In [60]:
dados

,store,day_of_week,date,sales,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval
0,965,7,2014-11-02,0,0,0,0,None,0,a,extended,110.0,8,2009,0,0,0,No promo2
1,268,1,2013-12-30,6995,814,1,0,None,1,a,basic,4520.0,2,2014,0,0,0,No promo2
2,1112,6,2015-03-28,8671,774,1,0,None,0,c,extended,1880.0,4,2006,0,0,0,No promo2
3,128,1,2014-07-21,6675,534,1,0,None,0,d,extended,2000.0,8,2009,1,1,2013,"Jan,Apr,Jul,Oct"
4,687,4,2015-06-11,6961,675,1,0,None,0,d,extended,2770.0,8,2009,0,0,0,No promo2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101716,410,1,2013-04-15,4233,618,1,0,None,0,c,basic,40.0,11,2011,1,22,2012,"Mar,Jun,Sept,Dec"
101717,1111,3,2014-12-03,6008,485,1,1,None,0,a,basic,1900.0,6,2014,1,31,2013,"Jan,Apr,Jul,Oct"
101718,262,2,2014-04-08,17914,3279,1,0,None,0,b,basic,1180.0,5,2013,0,0,0,No promo2
101719,849,6,2013-01-19,8214,922,1,0,None,0,c,extended,5000.0,8,2009,0,0,0,No promo2


In [62]:
# separar direito isso
dados_num = dados[['store','day_of_week','sales','customers','competition_distance','competition_open_since_month','competition_open_since_year','promo2_since_week','promo2_since_year']]
dados_cat = dados.drop(['store','day_of_week','sales','customers','competition_distance','competition_open_since_month','competition_open_since_year','promo2_since_week','promo2_since_year','date'],axis=1)

### 1.6.2 Numeric Variables Analysis

In [67]:
# Distribution in numbers

num_kurt = dados_num.apply(lambda x: x.kurtosis()) # Add kurtosis
num_skew = dados_num.apply(lambda x: x.skew()) # Add skew
num_range = dados_num.apply(lambda x: x.max() - x.min()) # Add range
pd.concat([dados_num.describe().T,num_kurt,num_skew,num_range],axis=1).rename(columns={0:'kurtosis',1:'skew',2:'range'}).drop('count',1)

,mean,std,min,25%,50%,75%,max,kurtosis,skew,range
store,559.726094,322.250508,1.0,281.0,559.0,839.0,1115.0,-1.204181,-0.002491,1114.0
day_of_week,3.998496,2.001751,1.0,2.0,4.0,6.0,7.0,-1.254069,0.001339,6.0
sales,5778.703041,3852.887607,0.0,3731.0,5745.0,7861.0,33913.0,1.858952,0.655340,33913.0
customers,634.492445,466.300820,0.0,405.0,610.0,838.0,5106.0,7.278215,1.631200,5106.0
competition_distance,7914.821649,50515.263406,20.0,710.0,2330.0,6890.0,999999.0,372.892805,19.145517,999979.0
competition_open_since_month,7.462717,2.696725,0.0,6.0,8.0,9.0,12.0,-0.348431,-0.497024,12.0
competition_open_since_year,2003.716381,100.964931,0.0,2008.0,2009.0,2011.0,2015.0,388.936828,-19.749031,2015.0
promo2_since_week,11.635483,15.333271,0.0,0.0,0.0,22.0,50.0,-0.496872,1.005324,50.0
promo2_since_year,1005.093147,1005.879672,0.0,0.0,0.0,2012.0,2015.0,-2.000031,0.001557,2015.0


Nothing strange.

The only strange values that we see were the ones that we chose arbitraly to substitute.

### 1.6.3 Categorical Variables Analysis

In [78]:
for column in dados_cat.columns.values:
    print(column.upper()+':')
    print(round(dados_cat[column].value_counts(normalize=True)*100,2))
    print('')

OPEN:
1    83.09
0    16.91
Name: open, dtype: float64

PROMO:
0    61.89
1    38.11
Name: promo, dtype: float64

STATE_HOLIDAY:
None              97.07
public holiday     1.91
Easter holiday     0.64
Christmas          0.38
Name: state_holiday, dtype: float64

SCHOOL_HOLIDAY:
0    82.19
1    17.81
Name: school_holiday, dtype: float64

STORE_TYPE:
a    54.20
d    30.74
c    13.44
b     1.62
Name: store_type, dtype: float64

ASSORTMENT:
basic       52.97
extended    46.17
extra        0.86
Name: assortment, dtype: float64

PROMO2:
0    50.04
1    49.96
Name: promo2, dtype: float64

PROMO_INTERVAL:
No promo2           50.04
Jan,Apr,Jul,Oct     28.85
Feb,May,Aug,Nov     11.67
Mar,Jun,Sept,Dec     9.45
Name: promo_interval, dtype: float64



Everything looks nice.

## 1.7 Columns and Values Filtering

In [83]:
# Let's filter what is useful and useless for our prediction based on the business demand
dados.drop('customers',axis=1,inplace=True) # We can't predict using this, because we don't have this answer to input
dados = dados[dados['open']==1] # We want to predict sales when the store is open. When it's closed, sales == 0
dados.drop('open',axis=1,inplace=True) # This column would be constant == 1, useless for our prediction

## 1.8 Finishing Data Cleaning

In [86]:
dados.to_csv('/Users/nando/Comunidade DS/ds_em_producao/data/data_cleaned.csv',index=False)

# 2.0 Exploratory Data Analysis

## 2.1 Import Data

In [3]:
dados = pd.read_csv('/Users/nando/Comunidade DS/ds_em_producao/data/data_cleaned.csv')

## 2.2 Hyphotesis

**DATE**

**1 -** As lojas vendem muito mais no começo do mês do que no final

**2 -** Houve um crescimento de vendas ao longo dos anos

**3 -** Lojas vendem mais no fim de semana

**4 -** Lojas vendem mais em feriados

**5 -** Lojas vendem mais no segundo semestre

**6 -** Lojas vendem mais no 2º e 3º trimestre.

**7 -** Lojas vendem mais no verão

**COMPETITION**

**8 -** Se a loja tem competidor ela vende menos

**9 -** Quanto mais velha é a competição, menos a loja vende

**10 -** Quanto mais longe é o competidor, mais vende a loja

**STORE**

**11 -** Existem tipos de loja que vendem substancialmente mais

**12 -** Quanto mais diversa é a loja, mais ela vende

**PROMOTIONS**

**13 -** Quando tem promoção as lojas vendem mais

**14 -** Quanto mais vezes se tem promoção em um período, menos eficaz ela fica

**15 -** Quanto mais vezes se tem promoção em um período, mais a loja vende

**16 -** Se a loja acatar a promo2 ela vende mais

**17 -** Existem períodos onde a promo2 é mais eficaz em relação às vendas

**18 -** Quanto mais antiga a promo2, menos a loja vende

## 2.3 Feature Engineering

In [98]:
# Day, Month, Week and Year Variables
dados['day'] = pd.to_datetime(dados['date']).dt.day
dados['month'] = pd.to_datetime(dados['date']).dt.month
dados['year'] = pd.to_datetime(dados['date']).dt.year
dados['week'] = pd.to_datetime(dados['date']).dt.week

In [40]:
# Semester and Quarter
dados['quarter'] = pd.to_datetime(dados['date']).dt.quarter
dados['semester'] = pd.to_datetime(dados['date']).dt.quarter.apply(lambda x: 1 if x == 1 or x == 2 else 2)

In [52]:
# Season
def season_of_date(date):
    year = str(date.year)
    seasons = {'spring': pd.date_range(start='21/03/'+year, end='20/06/'+year),
               'summer': pd.date_range(start='21/06/'+year, end='22/09/'+year),
               'autumn': pd.date_range(start='23/09/'+year, end='20/12/'+year)}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['autumn']:
        return 'autumn'
    else:
        return 'winter'

dados['season'] = pd.to_datetime(dados['date']).map(season_of_date)

In [85]:
# Competition Lifetime
dados['competition_lifetime'] = np.nan
for index in range(dados.shape[0]):
    try:
        data = datetime.datetime(dados.loc[index,'competition_open_since_year'],dados.loc[index,'competition_open_since_month'],1)
        dados.loc[index, 'competition_lifetime'] = (pd.to_datetime(dados.loc[index,'date']) - data).days
        
    except:
        dados.loc[index,'competition_lifetime'] = -999 # Never opened
dados['competition_lifetime'] = dados['competition_lifetime'].astype('int')

In [73]:
# Competition
dados['competition'] = dados['competition_open_since_month'].apply(lambda x: 0 if x == 0 else 1)

In [90]:
# Promo2 Lifetime
dados['promo2_lifetime'] = np.nan
for index in range(dados.shape[0]):
    if dados.loc[index,'promo_interval'] != 'No promo2':
        year = dados.loc[index,'promo2_since_year']
        week = dados.loc[index,'promo2_since_week']
        date = "{}-W{}".format(year,week)
        date_datetime = datetime.datetime.strptime(date + '-1', "%Y-W%W-%w")
        dados.loc[index,'promo2_lifetime'] = (pd.to_datetime(dados.loc[index,'date']) - date_datetime).days
    else:
        dados.loc[index,'promo2_lifetime'] = -999
dados['promo2_lifetime'] = dados['promo2_lifetime'].astype(int)

In [134]:
# Promo Count per Week
dados['promo_count_per_week'] = np.nan
for index in range(dados.shape[0]):
    week = dados.loc[index,'week']
    store = dados.loc[index,'store']
    try:
        dados.loc[index,'promo_count_per_week'] = dados[dados['store']==store].groupby('week').sum()['promo'][week]
    except:
        dados.loc[index,'promo_count_per_week'] = 0
dados['promo_count_per_week'] = dados['promo_count_per_week'].astype('int')

## 2.3 Finishing Feature Engineering

In [135]:
dados.to_csv('/Users/nando/Comunidade DS/ds_em_producao/data/data_feature_engineering')